# Server overview


In [6]:
use database_name


Commands completed successfully.

Total execution time: 00:00:00.030

## Indexes fragmentation

In [2]:
SELECT dbschemas.[name] as 'Schema',
  dbtables.[name] as 'Table',
  dbindexes.[name] as 'Index',
  indexstats.avg_fragmentation_in_percent,
  indexstats.page_count
FROM sys.dm_db_index_physical_stats (DB_ID(), NULL, NULL, NULL, NULL) AS indexstats
  INNER JOIN sys.tables dbtables on dbtables.[object_id] = indexstats.[object_id]
  INNER JOIN sys.schemas dbschemas on dbtables.[schema_id] = dbschemas.[schema_id]
  INNER JOIN sys.indexes AS dbindexes ON dbindexes.[object_id] = indexstats.[object_id]
    AND indexstats.index_id = dbindexes.index_id
WHERE indexstats.database_id = DB_ID()
AND indexstats.avg_fragmentation_in_percent > 10
ORDER BY indexstats.avg_fragmentation_in_percent desc


## Index usage

In [3]:
SELECT OBJECT_NAME(IX.OBJECT_ID) Table_Name
  , IX.name AS Index_Name
  , IX.type_desc Index_Type
  , SUM(PS.[used_page_count]) * 8 IndexSizeKB
  , IXUS.user_seeks AS NumOfSeeks
  , IXUS.user_scans AS NumOfScans
  , IXUS.user_lookups AS NumOfLookups
  , IXUS.user_updates AS NumOfUpdates
  , IXUS.last_user_seek AS LastSeek
  , IXUS.last_user_scan AS LastScan
  , IXUS.last_user_lookup AS LastLookup
  , IXUS.last_user_update AS LastUpdate
FROM sys.indexes IX
  INNER JOIN sys.dm_db_index_usage_stats IXUS ON IXUS.index_id = IX.index_id AND IXUS.OBJECT_ID = IX.OBJECT_ID
  INNER JOIN sys.dm_db_partition_stats PS on PS.object_id=IX.object_id
WHERE OBJECTPROPERTY(IX.OBJECT_ID,'IsUserTable') = 1
GROUP BY OBJECT_NAME(IX.OBJECT_ID) ,IX.name ,IX.type_desc ,IXUS.user_seeks ,IXUS.user_scans ,IXUS.user_lookups,IXUS.user_updates ,IXUS.last_user_seek ,IXUS.last_user_scan ,IXUS.last_user_lookup ,IXUS.last_user_update
ORDER BY IXUS.user_scans DESC

(0 rows affected)

Table_Name,Index_Name,Index_Type,IndexSizeKB,NumOfSeeks,NumOfScans,NumOfLookups,NumOfUpdates,LastSeek,LastScan,LastLookup,LastUpdate


Total execution time: 00:00:00.063

In [4]:
SELECT OBJECT_NAME(IX.OBJECT_ID) Table_Name
  , IX.name AS Index_Name
  , IX.type_desc Index_Type
  , SUM(PS.[used_page_count]) * 8 IndexSizeKB
  , IXUS.user_seeks AS NumOfSeeks
  , IXUS.user_scans AS NumOfScans
  , IXUS.user_lookups AS NumOfLookups
  , IXUS.user_updates AS NumOfUpdates
  , IXUS.last_user_seek AS LastSeek
  , IXUS.last_user_scan AS LastScan
  , IXUS.last_user_lookup AS LastLookup
  , IXUS.last_user_update AS LastUpdate
FROM sys.indexes IX
  INNER JOIN sys.dm_db_index_usage_stats IXUS ON IXUS.index_id = IX.index_id AND IXUS.OBJECT_ID = IX.OBJECT_ID
  INNER JOIN sys.dm_db_partition_stats PS on PS.object_id=IX.object_id
WHERE OBJECTPROPERTY(IX.OBJECT_ID,'IsUserTable') = 1
GROUP BY OBJECT_NAME(IX.OBJECT_ID) ,IX.name ,IX.type_desc ,IXUS.user_seeks ,IXUS.user_scans ,IXUS.user_lookups,IXUS.user_updates ,IXUS.last_user_seek ,IXUS.last_user_scan ,IXUS.last_user_lookup ,IXUS.last_user_update
ORDER BY (IXUS.user_scans + IXUS.user_seeks + IXUS.user_lookups ) DESC

(0 rows affected)

Table_Name,Index_Name,Index_Type,IndexSizeKB,NumOfSeeks,NumOfScans,NumOfLookups,NumOfUpdates,LastSeek,LastScan,LastLookup,LastUpdate


Total execution time: 00:00:00.057

# Disc Usage

In [5]:

WITH
  extra
  AS
  (
    -- Get info for FullText indexes, XML Indexes, etc
    SELECT sit.[object_id],
      sit.[parent_id],
      ps.[index_id],
      SUM(ps.reserved_page_count) AS [reserved_page_count],
      SUM(ps.used_page_count) AS [used_page_count]
    FROM sys.dm_db_partition_stats ps
      INNER JOIN sys.internal_tables sit
      ON  sit.[object_id] = ps.[object_id]
    WHERE   sit.internal_type IN
               (202, 204, 207, 211, 212, 213, 214, 215, 216, 221, 222, 236)
    GROUP BY    sit.[object_id],
                sit.[parent_id],
                ps.[index_id]
  ),
  agg
  AS
  (
    -- Get info for Tables, Indexed Views, etc (including "extra")
          SELECT ps.[object_id] AS [ObjectID],
        ps.index_id AS [IndexID],
        SUM(ps.in_row_data_page_count) AS [InRowDataPageCount],
        SUM(ps.used_page_count) AS [UsedPageCount],
        SUM(ps.reserved_page_count) AS [ReservedPageCount],
        SUM(ps.row_count) AS [RowCount],
        SUM(ps.lob_used_page_count + ps.row_overflow_used_page_count)
                    AS [LobAndRowOverflowUsedPageCount]
      FROM sys.dm_db_partition_stats ps
      GROUP BY    ps.[object_id],
                ps.[index_id]
    UNION ALL
      SELECT ex.[parent_id] AS [ObjectID],
        ex.[object_id] AS [IndexID],
        0 AS [InRowDataPageCount],
        SUM(ex.used_page_count) AS [UsedPageCount],
        SUM(ex.reserved_page_count) AS [ReservedPageCount],
        0 AS [RowCount],
        0 AS [LobAndRowOverflowUsedPageCount]
      FROM extra ex
      GROUP BY    ex.[parent_id],
                ex.[object_id]
  ),
  spaceused
  AS
  (
    SELECT agg.[ObjectID],
      OBJECT_SCHEMA_NAME(agg.[ObjectID]) AS [SchemaName],
      OBJECT_NAME(agg.[ObjectID]) AS [TableName],
      SUM(CASE
                WHEN (agg.IndexID < 2) THEN agg.[RowCount]
                ELSE 0
            END) AS [Rows],
      SUM(agg.ReservedPageCount) * 8 AS [ReservedKB],
      SUM(agg.LobAndRowOverflowUsedPageCount +
            CASE
                WHEN (agg.IndexID < 2) THEN (agg.InRowDataPageCount)
                ELSE 0
            END) * 8 AS [DataKB],
      SUM(agg.UsedPageCount - agg.LobAndRowOverflowUsedPageCount -
            CASE
                WHEN (agg.IndexID < 2) THEN agg.InRowDataPageCount
                ELSE 0
            END) * 8 AS [IndexKB],
      SUM(agg.ReservedPageCount - agg.UsedPageCount) * 8 AS [UnusedKB],
      SUM(agg.UsedPageCount) * 8 AS [UsedKB]
    FROM agg
    GROUP BY    agg.[ObjectID],
            OBJECT_SCHEMA_NAME(agg.[ObjectID]),
            OBJECT_NAME(agg.[ObjectID])
  )
SELECT sp.SchemaName,
  sp.TableName,
  sp.[Rows],
  sp.ReservedKB,
  (sp.ReservedKB / 1024.0 / 1024.0) AS [ReservedGB],
  sp.DataKB,
  (sp.DataKB / 1024.0 / 1024.0) AS [DataGB],
  sp.IndexKB,
  (sp.IndexKB / 1024.0 / 1024.0) AS [IndexGB],
  sp.UsedKB AS [UsedKB],
  (sp.UsedKB / 1024.0 / 1024.0) AS [UsedGB],
  sp.UnusedKB,
  (sp.UnusedKB / 1024.0 / 1024.0) AS [UnusedGB],
  so.[type_desc] AS [ObjectType],
  so.[schema_id] AS [SchemaID],
  sp.ObjectID
FROM spaceused sp
  INNER JOIN sys.all_objects so
  ON so.[object_id] = sp.ObjectID
WHERE so.is_ms_shipped = 0
--so.[name] LIKE N''  -- optional name filter
ORDER BY UsedGB DESC


(0 rows affected)

SchemaName,TableName,Rows,ReservedKB,ReservedGB,DataKB,DataGB,IndexKB,IndexGB,UsedKB,UsedGB,UnusedKB,UnusedGB,ObjectType,SchemaID,ObjectID


Total execution time: 00:00:00.097